# BMFH - Selbstlernender Roboter 🤖

![Roboter Klammern](openscad_klammern.png)

**Willkommen zum interaktiven Kurs!** In diesem Notebook lernen Sie, wie Sie einen elementaren Sensor-Aktor-Roboter programmieren, der sich selbst optimiert.

## 🎯 Kursziele
- Arduino-Programmierung mit Sensoren und Aktoren
- Grundlagen der Robotik-Steuerung
- Einführung in selbstlernende Algorithmen
- Praktische Umsetzung mit MPU-6050 und Servo-Motoren

## 🛠️ Hardware-Setup
- **Sensor**: MPU-6050 (XYZ-Lagesensor)
- **Aktoren**: 2x Servo-Motoren
- **Controller**: Arduino (mit Arduino IDE)
- **Stromversorgung**: Batterieblock für autonomen Betrieb

## 📚 Kursübersicht

| Lektion | Thema | Schwierigkeit | Dauer |
|---------|-------|---------------|-------|
| 1 | Diskrete Steuerung | ⭐ | 45 min |
| 2 | Positionslernen | ⭐⭐ | 60 min |
| 3 | Sinuskurven-Bewegung | ⭐⭐⭐ | 75 min |
| 4 | Intelligentes Lernen | ⭐⭐⭐⭐ | 90 min |

---
# 📖 Lektion 1: Kennenlernen der Steuerung

![Diskrete Steuerung](steuerungdiskret.png)

Wir beginnen mit **fixen Positionen**, die der Roboter ansteuern soll. Das ist der einfachste Einstieg in die Roboter-Programmierung.

## 🔧 Arduino-Grundcode (Diskrete Positionen)

Hier ist der Basis-Code für die erste Lektion:

In [ ]:
# Arduino Code (C++) - wird hier als Kommentar dargestellt
arduino_code_diskret = '''
#include <Servo.h>

Servo servoBox;  // Servo für Box-Bewegung
Servo servoFuss; // Servo für Fuss-Bewegung

// Fixe Positionen definieren
int positionen[][2] = {
  {90, 90},   // Startposition
  {120, 60},  // Position 1
  {60, 120},  // Position 2
  {90, 90}    // Zurück zu Start
};

void setup() {
  servoBox.attach(9);
  servoFuss.attach(10);
  Serial.begin(9600);
}

void loop() {
  for(int i = 0; i < 4; i++) {
    servoBox.write(positionen[i][0]);
    servoFuss.write(positionen[i][1]);
    delay(1000);
  }
}
'''

print("✅ Arduino-Code für diskrete Steuerung geladen")
print("📁 Vollständiger Code: ./kontinuierlich/kontinuierlich.ino")

## 🎮 Interaktive Simulation

Visualisieren wir die Servo-Bewegungen:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Diskrete Positionen simulieren
positionen = np.array([[90, 90], [120, 60], [60, 120], [90, 90]])
zeit = np.array([0, 1, 2, 3])

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6))

# Servo Box
ax1.step(zeit, positionen[:, 0], 'b-o', linewidth=2, markersize=8)
ax1.set_ylabel('Servo Box (Grad)')
ax1.set_title('🔧 Diskrete Servo-Steuerung')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(50, 130)

# Servo Fuss
ax2.step(zeit, positionen[:, 1], 'r-o', linewidth=2, markersize=8)
ax2.set_xlabel('Zeit (Sekunden)')
ax2.set_ylabel('Servo Fuss (Grad)')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(50, 130)

plt.tight_layout()
plt.show()

print("📊 Charakteristikum: Sprunghafte Bewegungen zwischen fixen Punkten")

---
# 🧠 Lektion 2: Lernen anhand der Positionen

Jetzt wird es spannend! Statt manuell Positionen festzulegen, **lernt** der Roboter mit Hilfe des MPU-6050 Sensors, ob er sich vorwärts bewegt.

## 📡 MPU-6050 Sensor Integration

In [ ]:
# Simulation des Lernalgorithmus
import random

class RoboterLernen:
    def __init__(self):
        self.beste_position = [90, 90]
        self.beste_bewegung = 0.0
        self.versuche = []
        
    def messe_bewegung(self, pos_box, pos_fuss):
        """Simuliert MPU-6050 Messung der Vorwärtsbewegung"""
        # Vereinfachte Simulation: Bewegung hängt von Position ab
        bewegung = np.sin(np.radians(pos_box)) * np.cos(np.radians(pos_fuss))
        bewegung += random.uniform(-0.1, 0.1)  # Sensor-Rauschen
        return max(0, bewegung)  # Nur positive Bewegung
    
    def lerne_position(self, max_versuche=20):
        """Genetischer Lernalgorithmus"""
        for versuch in range(max_versuche):
            # Zufällige Variation der aktuell besten Position
            neue_box = self.beste_position[0] + random.randint(-10, 10)
            neue_fuss = self.beste_position[1] + random.randint(-10, 10)
            
            # Begrenzungen
            neue_box = max(60, min(120, neue_box))
            neue_fuss = max(60, min(120, neue_fuss))
            
            # Teste neue Position
            bewegung = self.messe_bewegung(neue_box, neue_fuss)
            self.versuche.append(bewegung)
            
            # Wenn besser, übernehme neue Position
            if bewegung > self.beste_bewegung:
                self.beste_position = [neue_box, neue_fuss]
                self.beste_bewegung = bewegung
                print(f"🎯 Versuch {versuch+1}: Neue beste Position {self.beste_position}, Bewegung: {bewegung:.3f}")

# Lernprozess simulieren
roboter = RoboterLernen()
print("🚀 Starte Lernprozess...")
roboter.lerne_position()

print(f"\n✅ Optimale Position gefunden: {roboter.beste_position}")
print(f"📈 Beste Bewegung: {roboter.beste_bewegung:.3f}")

In [ ]:
# Lernfortschritt visualisieren
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(roboter.versuche, 'g-o', alpha=0.7)
plt.axhline(y=roboter.beste_bewegung, color='r', linestyle='--', label=f'Beste: {roboter.beste_bewegung:.3f}')
plt.xlabel('Versuch')
plt.ylabel('Gemessene Bewegung')
plt.title('🧠 Lernfortschritt')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# Bewegungslandschaft visualisieren
box_range = np.linspace(60, 120, 20)
fuss_range = np.linspace(60, 120, 20)
X, Y = np.meshgrid(box_range, fuss_range)
Z = np.sin(np.radians(X)) * np.cos(np.radians(Y))
Z = np.maximum(0, Z)  # Nur positive Werte

plt.contourf(X, Y, Z, levels=20, cmap='viridis')
plt.colorbar(label='Bewegung')
plt.plot(roboter.beste_position[0], roboter.beste_position[1], 'r*', markersize=15, label='Optimum')
plt.xlabel('Servo Box (Grad)')
plt.ylabel('Servo Fuss (Grad)')
plt.title('🗺️ Bewegungslandschaft')
plt.legend()

plt.tight_layout()
plt.show()

---
# 🌊 Lektion 3: Flüssigere Bewegungen mit Sinuskurven

![Sinuskurven-Steuerung](steuerungskurve.png)

Wechsel von fixen Positionen zu **Sinuskurven** für natürlichere, flüssigere Bewegungen.

In [ ]:
# Sinuskurven-basierte Bewegung
class SinusRoboter:
    def __init__(self):
        self.amplitude_box = 20    # Schwingungsbreite
        self.amplitude_fuss = 15
        self.frequenz_box = 1.0    # Schwingungsfrequenz
        self.frequenz_fuss = 1.2
        self.phase_box = 0         # Phasenverschiebung
        self.phase_fuss = np.pi/4
        
    def berechne_position(self, zeit):
        """Berechnet Servo-Positionen basierend auf Sinuskurven"""
        box_pos = 90 + self.amplitude_box * np.sin(2*np.pi*self.frequenz_box*zeit + self.phase_box)
        fuss_pos = 90 + self.amplitude_fuss * np.sin(2*np.pi*self.frequenz_fuss*zeit + self.phase_fuss)
        return box_pos, fuss_pos
    
    def simuliere_bewegung(self, dauer=5):
        """Simuliert Roboter-Bewegung über Zeit"""
        zeit = np.linspace(0, dauer, 100)
        positionen = []
        
        for t in zeit:
            box, fuss = self.berechne_position(t)
            positionen.append([box, fuss])
            
        return zeit, np.array(positionen)

# Simulation ausführen
sinus_roboter = SinusRoboter()
zeit, positionen = sinus_roboter.simuliere_bewegung()

# Visualisierung
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 8))

# Servo-Positionen über Zeit
ax1.plot(zeit, positionen[:, 0], 'b-', linewidth=2, label='Servo Box')
ax1.plot(zeit, positionen[:, 1], 'r-', linewidth=2, label='Servo Fuss')
ax1.set_ylabel('Position (Grad)')
ax1.set_title('🌊 Sinuskurven-Steuerung')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Phasendiagramm (Box vs Fuss)
ax2.plot(positionen[:, 0], positionen[:, 1], 'g-', linewidth=2)
ax2.set_xlabel('Servo Box (Grad)')
ax2.set_ylabel('Servo Fuss (Grad)')
ax2.set_title('🔄 Phasendiagramm (Bewegungsmuster)')
ax2.grid(True, alpha=0.3)
ax2.axis('equal')

# Geschwindigkeit (Ableitung)
geschwindigkeit = np.gradient(np.linalg.norm(np.gradient(positionen, axis=0), axis=1))
ax3.plot(zeit, geschwindigkeit, 'm-', linewidth=2)
ax3.set_xlabel('Zeit (s)')
ax3.set_ylabel('Geschwindigkeit')
ax3.set_title('⚡ Bewegungsgeschwindigkeit')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Sinuskurven erzeugen flüssige, natürliche Bewegungen")
print(f"📐 Phasenverschiebung: {sinus_roboter.phase_fuss/np.pi:.2f}π")

## 🎛️ Parameter-Optimierung

Jetzt optimieren wir die Sinuskurven-Parameter automatisch:

In [ ]:
class SinusOptimierung:
    def __init__(self):
        self.beste_parameter = {
            'amplitude_box': 20,
            'amplitude_fuss': 15,
            'frequenz_box': 1.0,
            'frequenz_fuss': 1.2,
            'phase_fuss': np.pi/4
        }
        self.beste_bewegung = 0
        
    def bewerte_parameter(self, parameter):
        """Bewertet Parameter-Set basierend auf simulierter Bewegung"""
        # Simulation der Bewegungseffizienz
        effizienz = (
            parameter['amplitude_box'] * parameter['frequenz_box'] +
            parameter['amplitude_fuss'] * parameter['frequenz_fuss'] +
            np.cos(parameter['phase_fuss']) * 10
        ) / 100
        
        # Zufälliges Rauschen für realistische Simulation
        effizienz += random.uniform(-0.1, 0.1)
        return max(0, effizienz)
    
    def optimiere(self, iterationen=15):
        """Evolutionärer Optimierungsalgorithmus"""
        verlauf = []
        
        for i in range(iterationen):
            # Zufällige Variation der Parameter
            neue_parameter = self.beste_parameter.copy()
            
            # Mutationen
            neue_parameter['amplitude_box'] += random.uniform(-5, 5)
            neue_parameter['amplitude_fuss'] += random.uniform(-3, 3)
            neue_parameter['frequenz_box'] += random.uniform(-0.3, 0.3)
            neue_parameter['frequenz_fuss'] += random.uniform(-0.3, 0.3)
            neue_parameter['phase_fuss'] += random.uniform(-0.5, 0.5)
            
            # Begrenzungen
            neue_parameter['amplitude_box'] = max(5, min(30, neue_parameter['amplitude_box']))
            neue_parameter['amplitude_fuss'] = max(5, min(25, neue_parameter['amplitude_fuss']))
            neue_parameter['frequenz_box'] = max(0.1, min(3, neue_parameter['frequenz_box']))
            neue_parameter['frequenz_fuss'] = max(0.1, min(3, neue_parameter['frequenz_fuss']))
            
            # Bewertung
            bewegung = self.bewerte_parameter(neue_parameter)
            verlauf.append(bewegung)
            
            if bewegung > self.beste_bewegung:
                self.beste_parameter = neue_parameter
                self.beste_bewegung = bewegung
                print(f"🎯 Iteration {i+1}: Verbesserte Parameter gefunden (Effizienz: {bewegung:.3f})")
        
        return verlauf

# Optimierung durchführen
optimierer = SinusOptimierung()
print("🚀 Starte Sinus-Parameter-Optimierung...")
verlauf = optimierer.optimiere()

print(f"\n✅ Optimierte Parameter:")
for key, value in optimierer.beste_parameter.items():
    print(f"   {key}: {value:.2f}")
print(f"📈 Beste Effizienz: {optimierer.beste_bewegung:.3f}")

# Optimierung visualisieren
plt.figure(figsize=(10, 4))
plt.plot(verlauf, 'o-', alpha=0.7)
plt.axhline(y=optimierer.beste_bewegung, color='r', linestyle='--', 
           label=f'Beste: {optimierer.beste_bewegung:.3f}')
plt.xlabel('Iteration')
plt.ylabel('Bewegungseffizienz')
plt.title('🧬 Evolutionäre Optimierung der Sinuskurven')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---
# 🧬 Lektion 4: Intelligenteres Lernen

Der fortgeschrittenste Ansatz: **Lokale Optimierung** in der Nähe bereits optimierter Werte. "Eingeschränkte Freiheitsgrade" für effizienteres Lernen.

In [ ]:
class IntelligentesLernen:
    def __init__(self, startwerte):
        self.parameter = startwerte.copy()
        self.parameter_historie = [startwerte]
        self.effizienz_historie = []
        self.lernrate = 0.1
        self.momentum = 0.0
        self.letzte_aenderung = {key: 0 for key in startwerte.keys()}
        
    def gradient_schaetzung(self, parameter_name):
        """Schätzt Gradienten durch finite Differenzen"""
        delta = 0.01
        
        # Aktuelle Effizienz
        aktuelle_effizienz = self.bewerte_parameter(self.parameter)
        
        # Parameter leicht ändern
        test_parameter = self.parameter.copy()
        test_parameter[parameter_name] += delta
        neue_effizienz = self.bewerte_parameter(test_parameter)
        
        # Gradient berechnen
        gradient = (neue_effizienz - aktuelle_effizienz) / delta
        return gradient
    
    def bewerte_parameter(self, parameter):
        """Erweiterte Bewertungsfunktion"""
        # Komplexere Fitnessfunktion
        amplitude_faktor = np.sqrt(parameter['amplitude_box'] * parameter['amplitude_fuss'])
        frequenz_faktor = parameter['frequenz_box'] * parameter['frequenz_fuss']
        phase_faktor = np.sin(parameter['phase_fuss']) + 1
        
        effizienz = (amplitude_faktor + frequenz_faktor * 5 + phase_faktor * 3) / 20
        
        # Bestrafung für extreme Werte
        if parameter['amplitude_box'] > 25 or parameter['amplitude_box'] < 10:
            effizienz *= 0.8
        if parameter['frequenz_box'] > 2 or parameter['frequenz_box'] < 0.5:
            effizienz *= 0.8
            
        effizienz += random.uniform(-0.05, 0.05)  # Rauschen
        return max(0, effizienz)
    
    def adaptives_lernen(self, iterationen=20):
        """Gradient-basiertes Lernen mit Momentum"""
        for i in range(iterationen):
            aktuelle_effizienz = self.bewerte_parameter(self.parameter)
            self.effizienz_historie.append(aktuelle_effizienz)
            
            # Für jeden Parameter Gradient berechnen
            neue_parameter = self.parameter.copy()
            
            for param_name in self.parameter.keys():
                gradient = self.gradient_schaetzung(param_name)
                
                # Momentum berücksichtigen
                aenderung = self.lernrate * gradient + self.momentum * self.letzte_aenderung[param_name]
                neue_parameter[param_name] += aenderung
                self.letzte_aenderung[param_name] = aenderung
            
            # Begrenzungen anwenden
            neue_parameter['amplitude_box'] = max(5, min(30, neue_parameter['amplitude_box']))
            neue_parameter['amplitude_fuss'] = max(5, min(25, neue_parameter['amplitude_fuss']))
            neue_parameter['frequenz_box'] = max(0.1, min(3, neue_parameter['frequenz_box']))
            neue_parameter['frequenz_fuss'] = max(0.1, min(3, neue_parameter['frequenz_fuss']))
            
            # Parameter übernehmen wenn besser
            neue_effizienz = self.bewerte_parameter(neue_parameter)
            if neue_effizienz > aktuelle_effizienz:
                self.parameter = neue_parameter
                self.parameter_historie.append(neue_parameter.copy())
                print(f"🎯 Iteration {i+1}: Verbesserung auf {neue_effizienz:.3f}")
            else:
                # Lernrate anpassen bei Verschlechterung
                self.lernrate *= 0.95
                self.momentum = min(0.9, self.momentum + 0.1)

# Intelligentes Lernen starten
startwerte = optimierer.beste_parameter  # Von vorheriger Optimierung
intelligenter_roboter = IntelligentesLernen(startwerte)

print("🧠 Starte intelligentes adaptives Lernen...")
print(f"📍 Startwerte: Effizienz = {intelligenter_roboter.bewerte_parameter(startwerte):.3f}")

intelligenter_roboter.adaptives_lernen()

print(f"\n🏆 Finale Parameter:")
for key, value in intelligenter_roboter.parameter.items():
    print(f"   {key}: {value:.3f}")
finale_effizienz = intelligenter_roboter.bewerte_parameter(intelligenter_roboter.parameter)
print(f"🚀 Finale Effizienz: {finale_effizienz:.3f}")

In [ ]:
# Lernfortschritt visualisieren
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Effizienz über Zeit
ax1.plot(intelligenter_roboter.effizienz_historie, 'b-o', alpha=0.7, linewidth=2)
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Effizienz')
ax1.set_title('🧠 Intelligentes Lernen - Effizienzentwicklung')
ax1.grid(True, alpha=0.3)

# Parameter-Evolution
if len(intelligenter_roboter.parameter_historie) > 1:
    param_evolution = {key: [] for key in intelligenter_roboter.parameter.keys()}
    for params in intelligenter_roboter.parameter_historie:
        for key, value in params.items():
            param_evolution[key].append(value)
    
    for key, values in param_evolution.items():
        if len(values) > 1:
            ax2.plot(values, label=key, linewidth=2, alpha=0.8)
    
    ax2.set_xlabel('Iteration')
    ax2.set_ylabel('Parameter-Wert')
    ax2.set_title('📈 Parameter-Evolution')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Vergleich der Lernmethoden
print("\n📊 Vergleich der Lernmethoden:")
print(f"🔧 Einfache Optimierung: {optimierer.beste_bewegung:.3f}")
print(f"🧠 Intelligentes Lernen: {finale_effizienz:.3f}")
verbesserung = ((finale_effizienz - optimierer.beste_bewegung) / optimierer.beste_bewegung) * 100
print(f"📈 Verbesserung: {verbesserung:.1f}%")

---
# 🎯 Kurs-Zusammenfassung

## ✅ Was Sie gelernt haben:

### 1. **Grundlagen der Roboter-Steuerung**
- Arduino-Programmierung mit Servos
- Sensor-Integration (MPU-6050)
- I2C-Kommunikation

### 2. **Evolutionäre Algorithmen**
- Zufällige Variation und Selektion
- Fitness-Bewertung
- Generationsbasierte Optimierung

### 3. **Mathematische Bewegungsmodelle**
- Sinuskurven für natürliche Bewegungen
- Phasenverschiebungen und Frequenzen
- Amplitude-Optimierung

### 4. **Intelligente Lernverfahren**
- Gradient-basierte Optimierung
- Momentum und adaptive Lernraten
- Lokale vs. globale Optimierung

## 🚀 Nächste Schritte:
- Hardware-Implementierung testen
- Verschiedene Terrains ausprobieren
- Multi-objektive Optimierung
- Neuronale Netze für komplexere Bewegungen

![Roboter Animation](fullmove.gif)

---
**🎓 Herzlichen Glückwunsch!** Sie haben erfolgreich einen selbstlernenden Roboter programmiert!